In [72]:
#importing library
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import tensorflow as tf

##Loading the data

In [73]:
df=pd.read_csv('../input/fifa-2018-match-statistics/FIFA 2018 Statistics.csv')

In [74]:
df

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11-07-2018,England,Croatia,1,46,11,1,6,4,4,...,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,14-07-2018,Belgium,England,2,43,12,4,3,5,4,...,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,14-07-2018,England,Belgium,0,57,15,5,7,3,5,...,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,15-07-2018,France,Croatia,4,39,8,6,1,1,2,...,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


# #Preprocessing

In [75]:
df=df.drop('Date',axis=1)


#Checking for missing value

In [76]:
df.isnull().sum()

Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [77]:
df=df.drop(['Own goals','Own goal Time'],axis=1)

In [78]:
df['1st Goal']=df['1st Goal'].fillna(df['1st Goal'].mean())

Encoding

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Team                    128 non-null    object 
 1   Opponent                128 non-null    object 
 2   Goal Scored             128 non-null    int64  
 3   Ball Possession %       128 non-null    int64  
 4   Attempts                128 non-null    int64  
 5   On-Target               128 non-null    int64  
 6   Off-Target              128 non-null    int64  
 7   Blocked                 128 non-null    int64  
 8   Corners                 128 non-null    int64  
 9   Offsides                128 non-null    int64  
 10  Free Kicks              128 non-null    int64  
 11  Saves                   128 non-null    int64  
 12  Pass Accuracy %         128 non-null    int64  
 13  Passes                  128 non-null    int64  
 14  Distance Covered (Kms)  128 non-null    in

In [80]:
df['Team'].unique()

array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Morocco', 'Iran',
       'Portugal', 'Spain', 'France', 'Australia', 'Argentina', 'Iceland',
       'Peru', 'Denmark', 'Croatia', 'Nigeria', 'Costa Rica', 'Serbia',
       'Germany', 'Mexico', 'Brazil', 'Switzerland', 'Sweden',
       'Korea Republic', 'Belgium', 'Panama', 'Tunisia', 'England',
       'Colombia', 'Japan', 'Poland', 'Senegal'], dtype=object)

In [81]:
label=LabelEncoder()
df['Man of the Match']=label.fit_transform(df['Man of the Match'])
man_mapping={index:label for index,label in enumerate(label.classes_)}

df['PSO']=label.fit_transform(df['PSO'])
pso_mapping={index:label for index,label in enumerate(label.classes_)}
print(pso_mapping)



{0: 'No', 1: 'Yes'}


In [82]:
df

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,...,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,...,118,22,0,0,0,1,12.000000,Group Stage,0,0
1,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,...,105,10,0,0,0,0,39.457447,Group Stage,0,0
2,Egypt,Uruguay,0,43,8,3,3,2,0,1,...,112,12,2,0,0,0,39.457447,Group Stage,0,0
3,Uruguay,Egypt,1,57,14,4,6,4,5,1,...,111,6,0,0,0,1,89.000000,Group Stage,0,0
4,Morocco,Iran,0,64,13,3,6,4,5,0,...,101,22,1,0,0,0,39.457447,Group Stage,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,Croatia,1,46,11,1,6,4,4,3,...,148,14,1,0,0,0,5.000000,Semi- Finals,0,0
124,Belgium,England,2,43,12,4,3,5,4,1,...,108,11,1,0,0,1,4.000000,3rd Place,0,0
125,England,Belgium,0,57,15,5,7,3,5,0,...,110,5,2,0,0,0,39.457447,3rd Place,0,0
126,France,Croatia,4,39,8,6,1,1,2,1,...,99,14,2,0,0,1,18.000000,Final,0,0


In [83]:
round=df['Round'].unique()
print(round)
round_mapping={label:index for index,label in enumerate(round)}
print(round_mapping)

['Group Stage' 'Round of 16' 'Quarter Finals' 'Semi- Finals' '3rd Place'
 'Final']
{'Group Stage': 0, 'Round of 16': 1, 'Quarter Finals': 2, 'Semi- Finals': 3, '3rd Place': 4, 'Final': 5}


In [84]:
df['Round']=df['Round'].apply(lambda x:round_mapping[x])


In [85]:
df['Team'].unique()

array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Morocco', 'Iran',
       'Portugal', 'Spain', 'France', 'Australia', 'Argentina', 'Iceland',
       'Peru', 'Denmark', 'Croatia', 'Nigeria', 'Costa Rica', 'Serbia',
       'Germany', 'Mexico', 'Brazil', 'Switzerland', 'Sweden',
       'Korea Republic', 'Belgium', 'Panama', 'Tunisia', 'England',
       'Colombia', 'Japan', 'Poland', 'Senegal'], dtype=object)

In [86]:
team=pd.get_dummies(df['Team'])


In [87]:
opponent=pd.get_dummies(df['Opponent'].apply(lambda x: 'opp_'+x))

In [88]:
df=pd.concat([df,team,opponent],axis=1)
df=df.drop(['Team','Opponent'],axis=1)


#Scaler

In [89]:
y=df['Man of the Match']
x=df.drop('Man of the Match',axis=1)

In [90]:
robust=RobustScaler()
x=pd.DataFrame(robust.fit_transform(x),columns=x.columns)
x

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,...,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,2.0,-0.6250,0.166667,1.166667,-0.666667,0.000000,0.333333,1.0,-0.571429,-0.666667,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.5,0.6250,-1.000000,-1.166667,-0.666667,0.000000,-1.000000,0.0,1.428571,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.5,-0.4375,-0.666667,-0.166667,-0.666667,-0.444444,-1.666667,0.0,-1.142857,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.4375,0.333333,0.166667,0.333333,0.444444,0.000000,0.0,-0.285714,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.5,0.8750,0.166667,-0.166667,0.333333,0.444444,0.000000,-0.5,-0.142857,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.0,-0.2500,-0.166667,-0.833333,0.333333,0.444444,-0.333333,1.0,1.285714,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.5,-0.4375,0.000000,0.166667,-0.666667,0.888889,-0.333333,0.0,-1.428571,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,-0.5,0.4375,0.500000,0.500000,0.666667,0.000000,0.000000,-0.5,-0.428571,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,1.5,-0.6875,-0.666667,0.833333,-1.333333,-0.888889,-1.000000,0.0,-0.142857,-0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# ##Spliting the data

In [91]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)


In [95]:
sk_model=MLPClassifier()
sk_model.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier()

In [96]:
sk_model.score(x_test,y_test)

0.6410256410256411

In [101]:
inputs=tf.keras.Input(shape=(85,))
x=tf.keras.layers.Dense(32,activation=tf.nn.relu)(inputs)
x=tf.keras.layers.Dense(32,activation=tf.nn.relu)(x)
outputs=tf.keras.layers.Dense(2,activation=tf.nn.softmax)(x)
tf_model=tf.keras.Model(inputs=inputs,outputs=outputs)

In [102]:
tf_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])

In [103]:
tf_model.fit(x_train,y_train,batch_size=16,epochs=200)

2022-01-10 13:16:30.241238: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 0.7060 - accuracy: 0.5393
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 0.6615 - accuracy: 0.5955
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 0.6337 - accuracy: 0.6966
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 0.6089 - accuracy: 0.7303
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 0.5845 - accuracy: 0.7640
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 0.5637 - accuracy: 0.8090
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 0.5396 - accuracy: 0.8202
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 0.5167 - accuracy: 0.8315
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.8427
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 0.4706 - accuracy: 0.8652
Epoch 11/200
6/6 [===========

In [105]:
tf_model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 6ms/step - loss: 2.2007 - accuracy: 0.5897


[2.200653314590454, 0.5897436141967773]

In [106]:
sk_model.score(x_test,y_test)

0.6410256410256411